In [135]:
import psycopg2
import json
import pandas as pd
from konlpy.tag import Okt, Kkma
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
import csv
import threading

In [161]:
# 기업 키워드 추출

# 여기를 바꿔주세요!
news_article_csv_filename = 'news_article_2.csv'
corp_keyword_csv_filename = 'corp_keyword_2.csv'


df = pd.read_csv(news_article_csv_filename)
kkma = Kkma() 
okt = Okt()
#불용어제거
stopwords = ['머니투데이' , "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자","아", "휴", "아이구", "대한", "이번",
             "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가", "기업", "트진", "위해",
            "지금", "말씀", "지난", "올해"]
corpnames = df['corpname'].unique()  
print("기업개수",len(corpnames), "개")
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []
corp_keyword_list = []


def split_sentences(text, start, end, result):
    kkma = Kkma()
    sentences = kkma.sentences(text[start:end])
    result.extend(sentences)

def text2sentences(text):
    # 스레드 개수 설정
    num_threads = 4
    text_length = len(text)
    chunk_size = text_length // num_threads

    threads = []
    result = []

    # 스레드 생성 및 실행
    for i in range(num_threads):
        start = i * chunk_size
        end = (i + 1) * chunk_size if i != num_threads - 1 else text_length
        thread = threading.Thread(target=split_sentences, args=(text, start, end, result))
        thread.start()
        threads.append(thread)

    # 모든 스레드 종료 대기
    for thread in threads:
        thread.join()

    return result


def get_nouns(corpname, sentences):
    nouns = []
    for sentence in sentences:
        if sentence != '':
            nouns.append(' '.join([noun for noun in okt.nouns(str(sentence))
                                  if noun not in stopwords and noun not in corpname and len(noun) > 1]))
    return nouns

def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

def get_ranks(graph, d=0.85): # d = damping factor
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0 # diagonal 부분을 0으로
        link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1

    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
    return {idx: r[0] for idx, r in enumerate(ranks)}


def keywords(word_num=20):

    keywords = []
    index=[]
    for idx in sorted_word_rank_idx[:word_num]:
        index.append(idx)

    #index.sort()
    for idx in index:
        keywords.append(idx2word[idx])
    
    return keywords

for corpname in corpnames:
    if len(corpname) > 15 :
        continue
    print(corpname)
    articles = df[df['corpname'] == corpname]['article']
    combined_articles = ' '.join(articles)
    sentences = text2sentences(combined_articles)
    nouns = get_nouns(corpname, sentences)
    words_graph, idx2word = build_words_graph(nouns)
    word_rank_idx = get_ranks(words_graph)
    sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)
    new_row = [corpname, keywords()]
    corp_keyword_list.append(new_row)
    print("완료 : ", corpname)

corp_keyword_df = pd.DataFrame(corp_keyword_list, columns=['corpname','keywords'])
corp_keyword_df.to_csv(corp_keyword_csv_filename, index=False, encoding='utf-8')

기업개수 847 개
솔루엠
완료 :  솔루엠
HK이노엔
완료 :  HK이노엔
한농화성
완료 :  한농화성
덕성
완료 :  덕성
동서
완료 :  동서
신도리코
완료 :  신도리코
상아프론테크
완료 :  상아프론테크
인베니아
완료 :  인베니아
가온칩스
완료 :  가온칩스
펄어비스
완료 :  펄어비스
세진중공업
완료 :  세진중공업
램테크놀러지
완료 :  램테크놀러지
서린바이오
완료 :  서린바이오
아세아시멘트
완료 :  아세아시멘트
동일기연
완료 :  동일기연
호텔신라
완료 :  호텔신라
부광약품
완료 :  부광약품
아모센스
완료 :  아모센스
한국정보통신
완료 :  한국정보통신
선익시스템
완료 :  선익시스템
한익스프레스
완료 :  한익스프레스
한국정밀기계
완료 :  한국정밀기계
핸디소프트
완료 :  핸디소프트
금화피에스시
완료 :  금화피에스시
플레이위드
완료 :  플레이위드
풍산
완료 :  풍산
삼성전기
완료 :  삼성전기
하이트진로홀딩스
완료 :  하이트진로홀딩스
이노션
완료 :  이노션
금호건설
완료 :  금호건설
파인엠텍
완료 :  파인엠텍
로지시스
완료 :  로지시스
대원제약
완료 :  대원제약
제이엘케이
완료 :  제이엘케이
LF
완료 :  LF
비츠로테크
완료 :  비츠로테크
사조산업
완료 :  사조산업
프리엠스
완료 :  프리엠스
SG세계물산
완료 :  SG세계물산
유니퀘스트
완료 :  유니퀘스트
동아화성
완료 :  동아화성
코스모화학
완료 :  코스모화학
유엔젤
완료 :  유엔젤
파커스
완료 :  파커스
경동도시가스
완료 :  경동도시가스
신진에스엠
완료 :  신진에스엠
경인전자
완료 :  경인전자
인피니트헬스케어
완료 :  인피니트헬스케어
인트론바이오
완료 :  인트론바이오
씨에스윈드
완료 :  씨에스윈드
에스씨디
완료 :  에스씨디
서울제약
완료 :  서울제약
지엔코
완료 :  지엔코
나노엔텍
완료 :  나노엔텍
정상제이엘에스
완료 :  정상제이엘에스
신흥
완료 :  신흥
한전기술
완료 :  한전기술
EMB
완료 :  EMB
미스터블루
완

완료 :  한라IMS
경동나비엔
완료 :  경동나비엔
한국가구
완료 :  한국가구
신화콘텍
완료 :  신화콘텍
LX인터내셔널
완료 :  LX인터내셔널
라이콤
완료 :  라이콤
성문전자
완료 :  성문전자
큐엠씨
완료 :  큐엠씨
엠게임
완료 :  엠게임
알테오젠
완료 :  알테오젠
엘리비젼
완료 :  엘리비젼
엘티씨
완료 :  엘티씨
미래에셋증권
완료 :  미래에셋증권
우주일렉트로
완료 :  우주일렉트로
코텍
완료 :  코텍
한미글로벌
완료 :  한미글로벌
넥스틴
완료 :  넥스틴
드림어스컴퍼니
완료 :  드림어스컴퍼니
크린앤사이언스
완료 :  크린앤사이언스
세명전기
완료 :  세명전기
안지오랩
완료 :  안지오랩
우신시스템
완료 :  우신시스템
KPX케미칼
완료 :  KPX케미칼
한국쉘석유
완료 :  한국쉘석유
베스파
완료 :  베스파
코맥스
완료 :  코맥스
백산
완료 :  백산
메디아나
완료 :  메디아나
지앤이헬스케어
완료 :  지앤이헬스케어
아난티
완료 :  아난티
노랑풍선
완료 :  노랑풍선
에스티오
완료 :  에스티오
대동스틸
완료 :  대동스틸
우성
완료 :  우성
KCI
완료 :  KCI
이녹스
완료 :  이녹스
신성델타테크
완료 :  신성델타테크
현대모비스
완료 :  현대모비스
JB금융지주
완료 :  JB금융지주
신일전자
완료 :  신일전자
인벤티지랩
완료 :  인벤티지랩
이수앱지스
완료 :  이수앱지스
나래나노텍
완료 :  나래나노텍
진양화학
완료 :  진양화학
휴엠앤씨
완료 :  휴엠앤씨
하이딥
완료 :  하이딥
에스앤더블류
완료 :  에스앤더블류
웰크론한텍
완료 :  웰크론한텍
피씨디렉트
완료 :  피씨디렉트
에스티큐브
완료 :  에스티큐브
디아이
완료 :  디아이
에이치케이
완료 :  에이치케이
대양금속
완료 :  대양금속
코이즈
완료 :  코이즈
서호전기
완료 :  서호전기
장원테크
완료 :  장원테크
모베이스전자
완료 :  모베이스전자
삼익악기
완료 :  삼익악기
동방아그로
완료 :  동방아그로
크리스에프앤씨
완료 :  크리스에프앤

In [ ]:
# 뉴스 키워드 및 3줄 요약 가져오기

# 여기를 바꿔주세요!
news_article_csv_filename = 'news_article_2.csv'
news_keyword_csv_filename = 'news_keyword_2.csv'


df = pd.read_csv(news_article_csv_filename)
rows = df.values.tolist()
kkma = Kkma() 
okt = Okt()
#불용어제거
stopwords = ['머니투데이' , "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자","아", "휴", "아이구", "대한", "이번",
             "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가", "기업", "트진", "위해",
            "지금", "말씀", "지난", "올해"]

tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []
news_keyword_list = []

def split_sentences(text, start, end, result):
    sentences = kkma.sentences(text[start:end])
    result.extend(sentences)

def text2sentences(text):
    # 스레드 개수 설정
    num_threads = 4
    text_length = len(text)
    chunk_size = text_length // num_threads

    threads = []
    result = []

    # 스레드 생성 및 실행
    for i in range(num_threads):
        start = i * chunk_size
        end = (i + 1) * chunk_size if i != num_threads - 1 else text_length
        thread = threading.Thread(target=split_sentences, args=(text, start, end, result))
        thread.start()
        threads.append(thread)

    # 모든 스레드 종료 대기
    for thread in threads:
        thread.join()

    return result

def get_nouns(corpname, sentences):
    nouns = []
    for sentence in sentences:
        if sentence != '':
            nouns.append(' '.join([noun for noun in okt.nouns(str(sentence))
                                  if noun not in stopwords and noun not in corpname and len(noun) > 1]))
    return nouns

def build_sent_graph(sentence):
    tfidf_mat = tfidf.fit_transform(sentence).toarray()
    graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return graph_sentence

def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

def get_ranks(graph, d=0.85): # d = damping factor
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0 # diagonal 부분을 0으로
        link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1

    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
    return {idx: r[0] for idx, r in enumerate(ranks)}

def summarize(sent_num=3):
    summary = []
    index=[]
    for idx in sorted_sent_rank_idx[:sent_num]:
        index.append(idx)

    index.sort()
#     print(index)
    
    for idx in index:
        summary.append(sentences[idx])

    return summary

def keywords(word_num=20):

    keywords = []
    index=[]
    for idx in sorted_word_rank_idx[:word_num]:
        index.append(idx)

    #index.sort()
    for idx in index:
        keywords.append(idx2word[idx])

    return keywords

print("기사 개수 :", len(rows))
a=1
for row in rows:
    corpname = row[0]
    print(a, corpname)
    link = row[1]
    article = row[2]
    sentences = text2sentences(article)
    nouns = get_nouns(corpname, sentences)
    try:
        sent_graph = build_sent_graph(nouns)
    except ValueError:
        print(f"{link} : ValueError", corpname)
        continue
    
    words_graph, idx2word = build_words_graph(nouns)
    sent_rank_idx = get_ranks(sent_graph)  #sent_graph : sentence 가중치 그래프
    sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k: sent_rank_idx[k], reverse=True)
    word_rank_idx = get_ranks(words_graph)
    sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)
    sum_sentence = summarize()
    sum_keyword = keywords()
    new_row = [corpname, link, sum_keyword, sum_sentence[0], sum_sentence[1], sum_sentence[2]]
    news_keyword_list.append(new_row)
    print("완료 : ", link)
    a+=1
news_keyword_df = pd.DataFrame(news_keyword_list, columns=['corpname', 'link', 'keywords', 'summary_sentence1', 'summary_sentence2', 'summary_sentence3'])
news_keyword_df.to_csv(news_keyword_csv_filename, index=False, encoding='utf-8')

기사 개수 : 48820
1 솔루엠
완료 :  https://www.itooza.com/common/iview.php?no=2023082309442565160
2 솔루엠
완료 :  https://www.itooza.com/common/iview.php?no=2023082308542955250
3 솔루엠
완료 :  https://www.thekpm.com/news/articleView.html?idxno=169807
4 솔루엠
완료 :  https://n.news.naver.com/mnews/article/009/0005175967?sid=101
5 솔루엠
완료 :  https://n.news.naver.com/mnews/article/011/0004229099?sid=101
6 솔루엠
완료 :  https://n.news.naver.com/mnews/article/015/0004882488?sid=101
7 솔루엠
완료 :  https://n.news.naver.com/mnews/article/018/0005555996?sid=101
8 솔루엠
완료 :  https://www.thekpm.com/news/articleView.html?idxno=169622
9 솔루엠
완료 :  https://n.news.naver.com/mnews/article/030/0003127847?sid=105
10 솔루엠
완료 :  https://n.news.naver.com/mnews/article/011/0004228690?sid=101
11 솔루엠
완료 :  http://www.newspim.com/news/view/20230821000828
12 솔루엠
완료 :  https://www.pinpointnews.co.kr/news/articleView.html?idxno=212951
13 솔루엠
완료 :  https://n.news.naver.com/mnews/article/015/0004881712?sid=101
14 솔루엠
완료 :  https://n.news.naver.co

완료 :  https://www.thekpm.com/news/articleView.html?idxno=168943
112 한농화성
완료 :  https://www.thekpm.com/news/articleView.html?idxno=168950
113 한농화성
완료 :  http://www.econonews.co.kr/news/articleView.html?idxno=299062
114 한농화성
완료 :  http://www.econonews.co.kr/news/articleView.html?idxno=298811
115 한농화성
완료 :  https://www.pinpointnews.co.kr/news/articleView.html?idxno=211909
116 한농화성
완료 :  http://www.econonews.co.kr/news/articleView.html?idxno=298565
117 한농화성
완료 :  http://www.econonews.co.kr/news/articleView.html?idxno=298385
118 한농화성
완료 :  http://www.econonews.co.kr/news/articleView.html?idxno=298211
119 한농화성
완료 :  https://www.pinpointnews.co.kr/news/articleView.html?idxno=211168
120 한농화성
완료 :  http://www.econonews.co.kr/news/articleView.html?idxno=298034
121 한농화성
완료 :  https://www.gukjenews.com/news/articleView.html?idxno=2782467
122 한농화성
완료 :  http://www.wsobi.com/news/articleView.html?idxno=210072
123 한농화성
완료 :  http://www.wsobi.com/news/articleView.html?idxno=210063
124 한농화성
완료 :  http:

완료 :  http://www.thebigdata.co.kr/view.php?ud=202308230627388758cd1e7f0bdf_23
220 덕성
완료 :  https://n.news.naver.com/mnews/article/003/0012045087?sid=101
221 덕성
완료 :  https://n.news.naver.com/mnews/article/018/0005556670?sid=101
222 덕성
완료 :  http://www.thebigdata.co.kr/view.php?ud=202308230529237350cd1e7f0bdf_23
223 덕성
완료 :  http://www.thebigdata.co.kr/view.php?ud=202308230524286393cd1e7f0bdf_23
224 동서
완료 :  https://n.news.naver.com/mnews/article/586/0000063312?sid=102
225 동서
완료 :  https://n.news.naver.com/mnews/article/119/0002742370?sid=101
226 동서
완료 :  https://n.news.naver.com/mnews/article/014/0005061120?sid=101
227 동서
완료 :  https://n.news.naver.com/mnews/article/092/0002302816?sid=105
228 동서
완료 :  https://n.news.naver.com/mnews/article/119/0002742356?sid=101
229 동서
완료 :  https://n.news.naver.com/mnews/article/421/0007006135?sid=102
230 동서
완료 :  https://n.news.naver.com/mnews/article/029/0002820960?sid=101
231 동서
완료 :  https://n.news.naver.com/mnews/article/079/0003804714?sid=102
23

완료 :  https://www.thekpm.com/news/articleView.html?idxno=168943
329 상아프론테크
완료 :  https://www.thekpm.com/news/articleView.html?idxno=168950
330 상아프론테크
완료 :  https://www.thekpm.com/news/articleView.html?idxno=168910
331 상아프론테크
완료 :  https://www.thekpm.com/news/articleView.html?idxno=168882
332 상아프론테크
완료 :  https://www.thekpm.com/news/articleView.html?idxno=168825
333 상아프론테크
완료 :  https://www.thekpm.com/news/articleView.html?idxno=168768
334 상아프론테크
완료 :  https://www.thekpm.com/news/articleView.html?idxno=168755
335 상아프론테크
완료 :  https://www.thekpm.com/news/articleView.html?idxno=168707
336 상아프론테크
완료 :  https://www.thekpm.com/news/articleView.html?idxno=168531
337 상아프론테크
완료 :  https://www.thekpm.com/news/articleView.html?idxno=168423
338 상아프론테크
완료 :  https://www.thekpm.com/news/articleView.html?idxno=168452
339 상아프론테크
완료 :  https://www.thekpm.com/news/articleView.html?idxno=168393
340 상아프론테크
완료 :  https://www.thekpm.com/news/articleView.html?idxno=168381
341 상아프론테크
완료 :  http://www.enetnews

완료 :  http://www.ggilbo.com/news/articleView.html?idxno=975461
435 인베니아
완료 :  http://www.lcnews.co.kr/news/articleView.html?idxno=52096
436 인베니아
완료 :  https://n.news.naver.com/mnews/article/421/0006816554?sid=101
437 인베니아
완료 :  http://www.ggilbo.com/news/articleView.html?idxno=975058
438 인베니아
완료 :  https://n.news.naver.com/mnews/article/123/0002305516?sid=101
439 인베니아
완료 :  https://www.pinpointnews.co.kr/news/articleView.html?idxno=196937
440 인베니아
완료 :  https://www.pinpointnews.co.kr/news/articleView.html?idxno=196926
441 인베니아
완료 :  http://www.econonews.co.kr/news/articleView.html?idxno=287569
442 인베니아
완료 :  http://www.ggilbo.com/news/articleView.html?idxno=974839
443 인베니아
완료 :  http://www.lcnews.co.kr/news/articleView.html?idxno=51866
444 인베니아
완료 :  https://www.pinpointnews.co.kr/news/articleView.html?idxno=196893
445 인베니아
완료 :  http://www.lcnews.co.kr/news/articleView.html?idxno=51863
446 인베니아
완료 :  https://www.itooza.com/common/iview.php?no=2023051715351219969
447 인베니아
완료 :  http://

...

완료 :  https://n.news.naver.com/mnews/article/009/0005171152?sid=101
21271 나노브릭
완료 :  https://www.thekpm.com/news/articleView.html?idxno=168150
21272 나노브릭
완료 :  https://n.news.naver.com/mnews/article/119/0002738700?sid=101
21273 나노브릭
완료 :  https://www.thekpm.com/news/articleView.html?idxno=167810
21274 나노브릭
완료 :  https://www.thekpm.com/news/articleView.html?idxno=167744
21275 나노브릭
완료 :  http://www.thebigdata.co.kr/view.php?ud=202308080702151316cd1e7f0bdf_23
21276 나노브릭
완료 :  http://www.ggilbo.com/news/articleView.html?idxno=987279
21277 나노브릭
완료 :  https://www.gukjenews.com/news/articleView.html?idxno=2782467
21278 나노브릭
완료 :  http://www.wsobi.com/news/articleView.html?idxno=210072
21279 나노브릭
완료 :  http://www.wsobi.com/news/articleView.html?idxno=210063
21280 나노브릭
완료 :  https://www.pinpointnews.co.kr/news/articleView.html?idxno=210918
21281 나노브릭
완료 :  https://www.cbci.co.kr/news/articleView.html?idxno=444626
21282 나노브릭
완료 :  http://www.thebigdata.co.kr/view.php?ud=20230807070648622cd1e7f0b

완료 :  https://n.news.naver.com/mnews/article/277/0005303683?sid=101
21377 현대건설
완료 :  https://sports.news.naver.com/news.nhn?oid=666&aid=0000019455
21378 현대건설
완료 :  https://www.thekpm.com/news/articleView.html?idxno=169927
21379 현대건설
완료 :  https://sports.news.naver.com/news.nhn?oid=421&aid=0007004673
21380 현대건설
완료 :  https://n.news.naver.com/mnews/article/018/0005556921?sid=102
21381 현대건설
완료 :  https://n.news.naver.com/mnews/article/277/0005303655?sid=101
21382 현대건설
완료 :  https://n.news.naver.com/mnews/article/011/0004229366?sid=101
21383 현대건설
완료 :  https://n.news.naver.com/mnews/article/421/0007004591?sid=102
21384 현대건설
완료 :  https://n.news.naver.com/mnews/article/382/0001070650?sid=101
21385 현대건설
완료 :  https://n.news.naver.com/mnews/article/003/0012045372?sid=101
21386 현대건설
완료 :  https://n.news.naver.com/mnews/article/015/0004882815?sid=101
21387 현대건설
완료 :  https://n.news.naver.com/mnews/article/366/0000926178?sid=101
21388 현대건설
완료 :  https://n.news.naver.com/mnews/article/011/0004229

완료 :  https://n.news.naver.com/mnews/article/079/0003784243?sid=102
21480 KH 건설
완료 :  https://www.news2day.co.kr/article/20230624500012
21481 KH 건설
완료 :  https://n.news.naver.com/mnews/article/018/0005515473?sid=101
21482 KH 건설
완료 :  https://n.news.naver.com/mnews/article/031/0000753570?sid=102
21483 KH 건설
완료 :  https://n.news.naver.com/mnews/article/008/0004902244?sid=102
21484 KH 건설
완료 :  https://n.news.naver.com/mnews/article/032/0003231172?sid=102
21485 KH 건설
완료 :  https://n.news.naver.com/mnews/article/079/0003781571?sid=102
21486 KH 건설
완료 :  https://n.news.naver.com/mnews/article/032/0003230640?sid=102
21487 KH 건설
완료 :  https://n.news.naver.com/mnews/article/032/0003230636?sid=102
21488 KH 건설
완료 :  https://n.news.naver.com/mnews/article/015/0004852982?sid=101
21489 KH 건설
완료 :  https://n.news.naver.com/mnews/article/018/0005501951?sid=105
21490 KH 건설
완료 :  https://n.news.naver.com/mnews/article/081/0003366401?sid=101
21491 KH 건설
완료 :  https://n.news.naver.com/mnews/article/016/000

완료 :  https://www.gukjenews.com/news/articleView.html?idxno=2767398
21585 한일사료
완료 :  https://www.news2day.co.kr/article/20230718500089
21586 한일사료
완료 :  https://www.thekpm.com/news/articleView.html?idxno=164290
21587 한일사료
완료 :  https://n.news.naver.com/mnews/article/009/0005160037?sid=101
21588 한일사료
완료 :  https://n.news.naver.com/mnews/article/015/0004869262?sid=101
21589 한일사료
완료 :  https://n.news.naver.com/mnews/article/018/0005532527?sid=101
21590 한일사료
완료 :  https://n.news.naver.com/mnews/article/003/0011980258?sid=101
21591 한일사료
완료 :  https://n.news.naver.com/mnews/article/215/0001114096?sid=101
21592 한일사료
완료 :  https://www.pinpointnews.co.kr/news/articleView.html?idxno=207534
21593 한일사료
완료 :  https://www.thekpm.com/news/articleView.html?idxno=164251
21594 한일사료
완료 :  https://www.cbci.co.kr/news/articleView.html?idxno=443125
21595 한일사료
완료 :  http://www.econonews.co.kr/news/articleView.html?idxno=295091
21596 한일사료
완료 :  http://www.thebigdata.co.kr/view.php?ud=202307180904597922cd1e7f0b

완료 :  https://n.news.naver.com/mnews/article/014/0005051170?sid=102
21689 인터지스
완료 :  https://www.thebell.co.kr/free/content/ArticleView.asp?key=202307062010221840106694
21690 인터지스
완료 :  https://n.news.naver.com/mnews/article/082/0001220459?sid=101
21691 인터지스
완료 :  https://n.news.naver.com/mnews/article/215/0001108828?sid=101
21692 인터지스
완료 :  https://www.pinpointnews.co.kr/news/articleView.html?idxno=202671
21693 인터지스
완료 :  https://www.news2day.co.kr/article/20230618500014
21694 인터지스
완료 :  http://www.wsobi.com/news/articleView.html?idxno=204186
21695 인터지스
완료 :  https://www.cbci.co.kr/news/articleView.html?idxno=441517
21696 인터지스
완료 :  http://www.thebigdata.co.kr/view.php?ud=2023061608260047107d270612f_23
21697 인터지스
완료 :  http://www.enetnews.co.kr/news/articleView.html?idxno=12632
21698 인터지스
완료 :  https://n.news.naver.com/mnews/article/421/0006843308?sid=102
21699 에너토크
완료 :  https://www.thekpm.com/news/articleView.html?idxno=169849
21700 에너토크
완료 :  https://n.news.naver.com/mnews/article/

In [19]:
#꼬꼬마분석기
#text를 입력받아 Kkma.sentences()를 이용해 문장단위로 나눈 뒤 sentences로 리턴
def text2sentences(text):  
    sentences = kkma.sentences(text)  #text일 때 문장별로 리스트 만듦
    for idx in range(0, len(sentences)):  #길이에 따라 문장 합침(위와 동일)
        if len(sentences[idx]) <= 10:
            sentences[idx-1] += (' ' + sentences[idx])
            sentences[idx] = ''
    return sentences

In [74]:
#불용어제거
stopwords = ['머니투데이' , "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자","아", "휴", "아이구", 
             "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가", "기업", "트진"]

def get_nouns(corpname, sentences):
    nouns = []
    for sentence in sentences:
        if sentence != '':
            nouns.append(' '.join([noun for noun in okt.nouns(str(sentence))
                                  if noun not in stopwords and noun not in corpname and len(noun) > 1]))
    return nouns

In [89]:
samsung = df[df['corpname'] == '삼성전자']

In [88]:
samsung

,corpname,link,article,pubdate
0,삼성전자,https://n.news.naver.com/mnews/article/015/000...,[ST기업 CEO 릴레이 인터뷰]③ 조현호 빌리어네어즈 대표토큰증권(ST) 기업 빌...,2023-08-20 22:25:00
1,삼성전자,https://sports.news.naver.com/news.nhn?oid=215...,"구글은 삼성전자, 카카오게임즈와 협업으로 출시된 '아레스: 라이즈 오브 가디언즈'가...",2023-08-20 21:52:00
2,삼성전자,https://n.news.naver.com/mnews/article/032/000...,콘텐츠·디바이스·운영체제 강화카카오게임즈는 신작 다중접속 역할수행게임(MMORPG)...,2023-08-20 21:49:00
3,삼성전자,https://n.news.naver.com/mnews/article/018/000...,[이데일리 김경은 기자] 다음은 21일자 미리보는 이데일리 신문 주요 뉴스다. △1...,2023-08-20 21:34:00
4,삼성전자,https://www.gukjenews.com/news/articleView.htm...,(용인=국제뉴스) 강정훈 기자 = 용인특례시(시장 이상일)는 ‘2023년 반도체 산...,2023-08-20 21:04:00
5,삼성전자,https://n.news.naver.com/mnews/article/032/000...,서울 여의도 증권가 인근 모습. 연합뉴스.지난해 기본급의 600%를 성과급으로 지급...,2023-08-20 21:00:00
6,삼성전자,https://n.news.naver.com/mnews/article/658/000...,"CEO스코어, 500대 중견기업 실적분석올해 상반기 국내 상장 중견기업의 영업이익이...",2023-08-20 20:53:00
7,삼성전자,https://n.news.naver.com/mnews/article/022/000...,22일 임시총회에서 명칭 변경1961년 설립 당시 이름 돌아가한경연 흡수 통합 방안...,2023-08-20 20:39:00
8,삼성전자,https://n.news.naver.com/mnews/article/656/000...,- 11월 세계전파통신회의 삼성전자 등 16개 기관 참여 정부는 2027년 차세...,2023-08-20 20:09:00
9,삼성전자,https://n.news.naver.com/mnews/article/016/000...,PC매장 상가[헤럴드경제= 박영훈 기자] “PC를 보고 있으면 그걸 사는 이유를 모...,2023-08-20 19:52:00


In [90]:
combined_string = ' '.join(samsung['article'])
corpname = '삼성전자'
sentences = text2sentences(combined_string)
nouns = get_nouns(corpname, sentences)
print(nouns)

['릴레이 인터뷰 대표 토큰 증권 어즈 조경 수목 토큰 증권 발행 준비', '나무 동안 아파트 단지 조성하 건설 업체 판매 수익금 토큰 투자자 배당 사업 모델', '조현 공동 대표 사진 방법 투자자 환산 중반 수익 예상', '대표 설명 수익 구조 다음', '먼저 네어즈 나무 농장 토지 임차', '네어즈 여기 지름 느티나무 산딸나무 왕벚나무 묘목 토큰 증권 투자자 판매 공모', '공모 가격 묘목 가격 판매 비용 판매 수반 비용 네어즈 취하 수수료 포함', '어즈 농장 필지 단위 농장 임차', '필지 보통 수천 그루 나무', '어즈 투자자 나무 그루 발행 조절 계획', '나무 그루 심겨 농장 유동 경우 발행 식이', '나무 보통 지름 건설 판매', '시간', '나무 판매 어즈 투자자 배당', '과정 투자자 배당금 수익률 환산 중반 대표 설명', '대표 조경 수목 중도 현금화 각종 변수 때문 농장 주가 미리 현금 사업 핵심 라며 나무 구매 측은 건설 사뿐 정부 중간 상인', '나무 입목 등기 부동산 신탁 방식 투자 자산 보호 장치 마련 계획', '사업 초기 유동화 나무 농장 면적 여의도 정도 넓이 대표 설명', '분야 업력 주식회사 협력 관계 시장 확보', '대표 최근 지속 나무 농장 조건 협력 의향 대부분 긍정 반응', '어즈 조경 수목 사업 기획 앞서 투자 플랫폼 리치 운영', '리치 이용자 플랫폼 다른 투자 고수 포트폴리오 대한 의견 포트폴리오 수도', '과거 대표 네이버 이수 대표 전자', '이후 가상 자산 거래소 이직 서로 의기 투합 창업', '리치 이용자 최근 돌파', '어즈 기사 투자', '투자 리치 운영 사업 기획 투자', '대표 네어즈 직접 농지 소유 묘목 계획', '회사 직접 농지 소유 수익 때문', '대해 대표 검토', '장의 수익 농지 임대 나무 시세 생산 현황 대한 데이터 확보', '대표 조경 수목 거래 데이터 조달청 공급 관련 인덱스 사업 계획 중장 기적 나무 탄소 배출 시장 조성하 목표', '수목 관련 탄소 배출 시장 나무 상응 배출 직접

In [77]:
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []

In [78]:
def build_sent_graph(sentence):
    tfidf_mat = tfidf.fit_transform(sentence).toarray()
    graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return graph_sentence

sent_graph = build_sent_graph(nouns)

In [79]:
def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
    
words_graph, idx2word = build_words_graph(nouns)

In [80]:
def get_ranks(graph, d=0.85): # d = damping factor
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0 # diagonal 부분을 0으로
        link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1

    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
    return {idx: r[0] for idx, r in enumerate(ranks)}

In [81]:
sent_rank_idx = get_ranks(sent_graph)  #sent_graph : sentence 가중치 그래프
sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k: sent_rank_idx[k], reverse=True)

In [82]:
word_rank_idx = get_ranks(words_graph)
sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)

In [83]:
def summarize(sent_num=5):
    summary = []
    index=[]
    for idx in sorted_sent_rank_idx[:sent_num]:
        index.append(idx)

    index.sort()
#     print(index)
    
    for idx in index:
        summary.append(sentences[idx])

    for text in summary :
        print(text)
        print("\n")

In [84]:
def keywords(word_num=10):

    keywords = []
    index=[]
    for idx in sorted_word_rank_idx[:word_num]:
        index.append(idx)

    #index.sort()
    for idx in index:
        keywords.append(idx2word[idx])

    print(keywords)

In [85]:
summarize()

━ 참 이슬 소주 8년 만에 반기 매출 감소… 주 종 다변화 추세에 경쟁사 제품 추격 ━22 일 하이 트진 로 반기보고서에 따르면 올해 상반기 연결 기준 소주 매출은 7364억원으로 지난해 상반기 (7450 억원) 대비 1.1% 감소했다.


올해 상반기 롯데 칠성 음료의 소주 매출액은 2045억원으로 지난해 상반기보다 22.7% 증가했다.


반면 국내 맥주 시장 1위 오비 맥주는 올해 상반기 국내 매출이 감소한 것으로 추정된다.


소주 시장에서 선전한 롯데 칠성 음료도 올해 상반기 맥주 사업은 부진했다.


올해 상반기 롯데 칠성 음료의 맥주 매출은 412억원으로 지난해 상반기 (512 억원 )보다 약 20% 감소했다.




In [86]:
keywords()

['매출', '경쟁', '소주', '상반기', '감소', '시장', '효과', '제품', '오비', '관계자']


In [152]:
# Redshift 연결
file_path = "./connections.json"

with open(file_path, 'r') as file:
    con = json.load(file)

# Redshift 연결 정보
host = con['redshift_conn']['host']
dbname = 'dev'
user = con['redshift_conn']['login']
password = con['redshift_conn']['password']
port = '5439'

# 쿼리 작성
query = 'SELECT corpname FROM "raw_data"."news_article";'

# Redshift에 연결
conn = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password,
    port=port
)
# 쿼리 실행 및 결과 가져오기
try:
    '''
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    '''
    df = pd.read_sql_query(query, conn)
    print(df)
except Exception as e:
    print("Error:", e)
finally:
    if conn:
        conn.close()

C:\Users\USER\AppData\Local\Temp\ipykernel_14312\688948444.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


       corpname
0          켐트로스
1          켐트로스
2          켐트로스
3          켐트로스
4          켐트로스
...         ...
144748    이원컴포텍
144749    이원컴포텍
144750    이원컴포텍
144751    이원컴포텍
144752    이원컴포텍

[144753 rows x 1 columns]


In [156]:
corpnames = list(df['corpname'].unique())

In [160]:
for corp in corpnames:
    if len(corp) < 10:
        continue
    print(len(corp))

10
10
10
10
10
10
10
11
10
10
10
10
13
10
11
10
10
14
10
10
10
12
11
11
